In [4]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = 5
    K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = 5
    K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_m(precision, recall):
    f1 = 2 * ([precision*recall]/precision+recall)
    return f1

In [5]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib


In [6]:
#Importando imagens

data_dir = pathlib.Path("../datasets/sign_lang/")
image_count = len(list(data_dir.glob('*/*.jpg')))
print(f"{image_count} images found")
A = list(data_dir.glob('A/*'))
B = list(data_dir.glob('B/*'))
C = list(data_dir.glob('C/*'))
D = list(data_dir.glob('D/*'))
E = list(data_dir.glob('E/*'))
F = list(data_dir.glob('F/*'))
G = list(data_dir.glob('G/*'))
H = list(data_dir.glob('H/*'))
I = list(data_dir.glob('I/*'))
J = list(data_dir.glob('J/*'))
K = list(data_dir.glob('K/*'))
L = list(data_dir.glob('L/*'))
M = list(data_dir.glob('M/*'))
N = list(data_dir.glob('N/*'))
O = list(data_dir.glob('O/*'))
P = list(data_dir.glob('P/*'))
Q = list(data_dir.glob('Q/*'))
R = list(data_dir.glob('R/*'))
S = list(data_dir.glob('S/*'))
T = list(data_dir.glob('T/*'))
U = list(data_dir.glob('U/*'))
V = list(data_dir.glob('V/*'))
W = list(data_dir.glob('W/*'))
X = list(data_dir.glob('X/*'))
Y = list(data_dir.glob('Y/*'))
Z = list(data_dir.glob('Z/*'))

#Definindo algumas variáveis

batch_size = 10 #total de 5 batches
img_height = 32
img_width = 32

390 images found


In [7]:
#Criando dataset de treino

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.34,
  subset="training",
  label_mode='categorical',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


#Criando dataset de test

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.34,
  subset="validation",
  label_mode='categorical',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#Deninindo o número de classes
classes = train_ds.class_names
lenClasses = len(classes)

Found 390 files belonging to 26 classes.
Using 258 files for training.


2022-06-30 11:13:57.800811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-30 11:13:57.800848: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-30 11:13:57.800879: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arch): /proc/driver/nvidia/version does not exist
2022-06-30 11:13:57.815666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 390 files belonging to 26 classes.
Using 132 files for validation.


In [8]:
# Aplicando algumas mudanças no dataset, para melhorar o funcionamento da rede neural
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [9]:
#Teste em um modelo padrão

model = tf.keras.Sequential()

model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1024, activation='sigmoid'))
model.add(tf.keras.layers.Dense(lenClasses, activation='softmax'))

model.compile(
  optimizer='adam',
  loss='categorical_crossentropy', 
  metrics=['accuracy',
           tf.keras.metrics.Recall(),
           tf.keras.metrics.Precision(),
          tf.keras.metrics.FalsePositives(),
          tf.keras.metrics.FalseNegatives(),
          tf.keras.metrics.TrueNegatives(),
          tf.keras.metrics.TruePositives()])

#Treinando
Nepochs = 10
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=Nepochs
)

#Testando
results = model.evaluate(val_ds)
print(f"Results with {Nepochs} training epochs:")
print(f"Accuracy: {results[1]}")
print(f"Recall: {results[2]}")
print(f"Precision: {results[3]}")


Epoch 1/10
26/26 [==============================] - 5s 81ms/step - loss: 3.2126 - accuracy: 0.1163 - recall: 0.0116 - precision: 0.6000 - false_positives: 2.0000 - false_negatives: 255.0000 - true_negatives: 6448.0000 - true_positives: 3.0000 - val_loss: 3.4533 - val_accuracy: 0.0455 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_false_positives: 0.0000e+00 - val_false_negatives: 132.0000 - val_true_negatives: 3300.0000 - val_true_positives: 0.0000e+00
Epoch 2/10
26/26 [==============================] - 1s 35ms/step - loss: 1.8346 - accuracy: 0.4612 - recall: 0.2132 - precision: 0.7857 - false_positives: 15.0000 - false_negatives: 203.0000 - true_negatives: 6435.0000 - true_positives: 55.0000 - val_loss: 3.2229 - val_accuracy: 0.0379 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_false_positives: 0.0000e+00 - val_false_negatives: 132.0000 - val_true_negatives: 3300.0000 - val_true_positives: 0.0000e+00
Epoch 3/10
26/26 [==============================] - 1s 34ms/

In [10]:
prediction = model.predict(val_ds)

14/14 [==============================] - 0s 7ms/step
